In [1]:
import os
print(os.listdir("../input"))
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

['images', 'test.csv', 'sample_submission.csv', 'train.csv']


In [2]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import time
import math
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.data import Dataset
from tqdm import tqdm
from sklearn import metrics
import torch.nn.functional as F
import random
import sys
import shutil
import albumentations
from albumentations import pytorch as AT

from efficientnet_pytorch import EfficientNet

In [3]:
import scipy.special
sigmoid = lambda x: scipy.special.expit(x)

SEED = 323
base_dir = '../input/'

def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(SEED)

In [4]:
sub = pd.read_csv( base_dir + 'sample_submission.csv')

In [5]:
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.25,0.25,0.25,0.25
1,Test_1,0.25,0.25,0.25,0.25
2,Test_2,0.25,0.25,0.25,0.25
3,Test_3,0.25,0.25,0.25,0.25
4,Test_4,0.25,0.25,0.25,0.25


In [6]:
class PlantDataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):

        row = self.df.iloc[idx]
        path = base_dir + '/images/' + row.image_id + '.jpg'
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)
            
        image = image['image']
            
        return image

In [7]:
def pre_trans(x, cols, rows):
    return (x * 2.0 - 1.0)

# test_transform = albumentations.Compose([
#     albumentations.SmallestMaxSize(IMG_SIZE),
#     albumentations.CenterCrop(IMG_SIZE, IMG_SIZE),
#     albumentations.Lambda(image = pre_trans),
#     AT.ToTensor(),
#     ])

In [8]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)
    
def mish(input):
    return input * torch.tanh(F.softplus(input))
       
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return mish(input)

In [9]:
TTA = 1
BATCH_SIZE = 64
all_pred = []

In [10]:
################
EXP = 1
FOLD = 5
IMG_SIZE = 320
################

tta_transform = albumentations.Compose([
    albumentations.Resize(IMG_SIZE, IMG_SIZE),
#     albumentations.RandomRotate90(p=0.5),
#     albumentations.Transpose(p=0.5),
#     albumentations.Flip(p=0.5),
    albumentations.Lambda(image = pre_trans),
    AT.ToTensor(),
    ])

testset       = PlantDataset(sub, transform = tta_transform)
test_loader   = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

pred_list = []
for fold in range(FOLD):
    print('*********Fold%d************'%fold)
    ############################################
    model_conv = EfficientNet.from_name('efficientnet-b5')
    model_conv._dropout = nn.Dropout(p=0.5)
    model_conv._avg_pooling = AdaptiveConcatPool2d()
    model_conv._fc = nn.Sequential(nn.Linear(2048*2,256), Mish(), nn.Dropout(p=0.5), nn.Linear(256,4))

    model_conv.load_state_dict(torch.load('./exp/exp' + str(EXP) + '/efficientnet-b5-best' + str(fold) + '.pth'))
    
    model_conv.cuda()
    model_conv.eval()
    ############################################
    for tta in range(TTA):
        y_pred_val = np.zeros((len(sub), 4))
        with torch.no_grad():
            for idx, imgs in enumerate(test_loader):
                imgs = imgs.cuda()
                output_test = model_conv(imgs) 
                b = output_test.detach().cpu().numpy()
                y_pred_val[idx*BATCH_SIZE:idx*BATCH_SIZE+b.shape[0]] = b

        pred_list.append(y_pred_val)

pred = sum(pred_list)/len(pred_list)
all_pred.append(pred)

/home/fangxi/miniconda3/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2908: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  "Using lambda is incompatible with multiprocessing. "


*********Fold0************
*********Fold1************
*********Fold2************
*********Fold3************
*********Fold4************


In [11]:
pred = all_pred[0] * 1
print(pred)

[[ -2.98558373   1.91002103   9.70751905  -6.03564281]
 [ -3.82836681   1.65191276  11.44217634  -7.01547756]
 [ -4.31053615  -0.0322613  -11.53221827  20.55994968]
 ...
 [ -1.71773199   0.88526987   8.77769222  -6.09365592]
 [  7.7611371   -1.99026644  -3.17295842  -1.93522797]
 [ -2.34323726   1.21094275  -6.64863539  10.90775356]]


In [12]:
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.25,0.25,0.25,0.25
1,Test_1,0.25,0.25,0.25,0.25
2,Test_2,0.25,0.25,0.25,0.25
3,Test_3,0.25,0.25,0.25,0.25
4,Test_4,0.25,0.25,0.25,0.25


In [13]:
def softmax(X, theta = 1.0, axis = None):

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter,
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis = axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis = axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

In [14]:
# for i, row in enumerate(pred):
#     if pred[i][1] > 0.5:
#         pred[i][2] -= pred[i][1]
#         pred[i][3] -= pred[i][1]
pred = softmax(pred, axis=1)
print(pred)

[[3.07097124e-06 4.10591433e-04 9.99586192e-01 1.45429302e-07]
 [2.33379632e-07 5.59909875e-05 9.99943766e-01 9.63647808e-09]
 [1.58083026e-11 1.14003031e-09 1.15491108e-14 9.99999999e-01]
 ...
 [2.76516349e-05 3.73413622e-04 9.99598587e-01 3.47761257e-07]
 [9.99862460e-01 5.82048953e-05 1.78370451e-05 6.14982015e-05]
 [1.75849174e-06 6.14753626e-05 2.37317456e-08 9.99936742e-01]]


In [15]:
sub['healthy'] = pred[:,0]
sub['multiple_diseases'] = pred[:,1]
sub['rust'] = pred[:,2]
sub['scab'] = pred[:,3]
sub.to_csv('submission.csv', index=False)
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,3.070971e-06,4.105914e-04,9.995862e-01,1.454293e-07
1,Test_1,2.333796e-07,5.599099e-05,9.999438e-01,9.636478e-09
2,Test_2,1.580830e-11,1.140030e-09,1.154911e-14,1.000000e+00
3,Test_3,9.999934e-01,2.023646e-06,9.819839e-07,3.566827e-06
4,Test_4,1.839029e-07,2.613559e-05,9.999737e-01,5.836191e-09
